# Data retrival

In [1]:
%load_ext autotime
%load_ext autoreload
%autoreload 2
from datetime import datetime
startTime = datetime.now()
# total time
# print datetime.now() - startTime 

In [2]:
import myfunctions as f
f.today()

'20180409'

time: 207 ms


In [3]:
# python version
import sys
print "system_info: %s"%sys.version
# current working directory
import os
print "path_info: %s"%os.getcwd() 

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue/Documentation_201712
time: 13.5 ms


In [4]:
websites = f.delivery_para()

time: 16.1 ms


> Timing

    fp - 7min 44s
    de - 5min 35s
    wte - 1min 26s

In [6]:
i = 2
dfs = []

time: 19.2 ms


In [9]:
df = f.process_rec(websites[i])
i = i-1
dfs.append(df)

time: 1min 26s


In [10]:
df = f.process_rec(websites[i])
i = i-1
dfs.append(df)

time: 5min 35s


In [11]:
df = f.process_rec(websites[i])
dfs.append(df)

time: 7min 44s


In [13]:
import pandas as pd
df1 = pd.concat(dfs)
df1.reset_index(inplace=True, drop=True)

time: 3.11 s


In [14]:
print df1.shape
print df1.columns.tolist()
df1.sample(2,random_state=2008)

(4048605, 13)
['address', 'desc', 'food_name', 'loc', 'neighbourhood', 'opening', 'phone', 'price', 'rating', 'restaurant.cuisine', 'tag', 'timestamp', 'vendor_name']


,address,desc,food_name,loc,neighbourhood,opening,phone,price,rating,restaurant.cuisine,tag,timestamp,vendor_name
2199073,NaN,NaN,Calamansi Asam Boi Juice,foodpanda/menu_item/54301,NaN,NaN,NaN,6.9,4.5,"[Halal, Malaysian]",Beverages,2017-07-11 01:13:18.670689,Agrobazaar Malaysia
1063485,"370 Alexandra Road, #B1-08, Singapore, 159953",NaN,"CH24 VEUVE MOISANS ""Blanc de Blancs"" - Loire V...",deliveroo/food/AWCwedZUvFWOHWF15nx0,Queensway,Open until 22:00,+6564768997,30.0,NaN,Wine,Sparkling Wine,2018-01-01T04:00:05.703635,Wine Connection Shop@Anchorpoint - Anchorpoint


time: 4.11 s


** Save food data with all details **

# Restaurant entities

In [16]:
restaurant = f.get_restaurant_details(df1)

time: 15.8 s


In [19]:
print restaurant.shape
print restaurant.columns.tolist()
restaurant.sample(2, random_state=2008)

(7101, 9)
['vendor_name', 'rating', 'opening', 'loc', 'crawling_range', 'address', 'phone', 'neighbourhood', 'restaurant.cuisine']


,vendor_name,rating,opening,loc,crawling_range,address,phone,neighbourhood,restaurant.cuisine
index,,,,,,,,,
delivery_2584,Jack's Place - Clementi,NaN,"[Open until 22:30, Opens at 11:30]","(deliveroo/food/157751, deliveroo/food/157756,...","[2017-07-03, 2018-04-01]","[12 West Coast Walk #01-04/05/06, Singapore, 1...",[+6567762775],[West Coast],"[Healthier Choices, Western]"
delivery_1735,Fix Café & Grill,NaN,"[Open until 20:30, Opens at 11:30]","(deliveroo/food/158930, deliveroo/food/158935,...","[2017-07-03, 2018-02-01]","[31 Ah Hood Road, #01-06 HomeTeam NS-JOM Clubh...",[+6562561484],[Whampoa],[Western]


time: 46.8 ms


In [20]:
## save to json
file_name = "__".join(["restaurant_entities", f.today()]) + ".json"
restaurant.to_json(file_name, orient='index')
print "saved: %s"%file_name

saved: restaurant_entities__20180409.json
time: 1.8 s


# Food entities

In [21]:
# ref dict: restaurant name : restaurant index
restaurant.reset_index(inplace=True)
restaurant_ref = restaurant.set_index(["vendor_name"]).to_dict()["index"]

time: 47.6 ms


In [26]:
food_items = f.get_food_entities(df1, restaurant_ref)

with 209680 unique menu names in 4048605 records
time: 1min 14s


In [27]:
print food_items.shape
print food_items.columns.tolist()
food_items.sample(2, random_state=2008)

(143973, 8)
['clean_name', 'loc', 'food_name', 'timestamp', 'price', 'vendors', 'tag', 'desc']


,clean_name,loc,food_name,timestamp,price,vendors,tag,desc
index,,,,,,,,
food_53819,grill pepper chicken,"[foodpanda/menu_item/10850, foodpanda/menu_ite...","[(W48. Grill Pepper Chicken, 82)]","[2017-07-11 01:13:18.670689, 2017-07-15 00:25:...","(9.63, 9.63, 9.63, 9.63, 9.63, 9.63, 9.63, 9.6...",[delivery_202],[Western],"[nan, Default]"
food_131936,the roots,"[deliveroo/food/26634, deliveroo/food/AV06vmHe...","[(The Roots, 10)]","[2017-07-03T17:55:01.668428, 2017-07-13T14:37:...","(16.05, 16.05, 16.05, 16.05, 16.05, 16.05, 16....","[delivery_6137, delivery_6138]","[Burgers, Burgers ]","[Seasonal vegetables, cheese fritter, coleslaw..."


time: 125 ms


In [28]:
## save to p file
file_name = "__".join(["food_entities", f.today()]) + ".p"
f.save_file(food_items, file_name)

saved: food_entities__20180409.p
time: 59.4 s


** construct test case **

In [30]:
# save test case to csv
file_name = "001_entity_identification_test_case_" + f.today()+".csv"
test_case = food_items.sample(100, random_state=2008)[["food_name", "clean_name"]]
test_case["food_name"] = test_case["food_name"].apply(lambda s: f.get_string(s[0][0]))
test_case.to_csv(file_name,index=False)
print "saved: %s"%file_name

saved: 001_entity_identification_test_case_20180409.csv
time: 39.5 ms
